Code for calculating dF/F for identified neuronal activity sources

@uthor: Raju Tomer

In [1]:
import os, time
import glob

import monnet_utils as tl

import skimage.external.tifffile as tff
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import re
import pickle
import scipy as spy
from scipy import ndimage as ndi

In [ ]:
# Load data
prefix_swap_sw = False

analysis_dir = r'X:\People\Raju\Batch06a'
data_dir = r'Y:\Data'
f_data_dir = r'fdata'
dict_in_fn = tl.load_pickle(f_data_dir, r'dict_in_fn.pickle')
dict_im_lab = tl.load_pickle(f_data_dir,  r'dict_im_lab.pickle')
xsaZ
for key in dict_in_fn.keys():
        max_file_name = os.path.basename(dict_in_fn[key])
        if ('MAX_' not in max_file_name):
            max_file_name = 'MAX_' + max_file_name
        dict_in_fn[key] = os.path.join(data_dir, max_file_name)
        print(dict_in_fn[key])

After treatment data
dict_in_fn_AT = {}
data_dir = r'Y:\Data\Aligned'

file_names = sorted(glob.glob(os.path.join(data_dir, 'MAX_*.tif')))
for i in range(0,len(file_names)):
    dict_in_fn_AT[i] = file_names[i]



In [ ]:
### to construct dict with key file names
file_names_A = sorted(glob.glob(os.path.join(analysis_dir, 'cnm_A_key*')))
print(file_names_A[0])
dict_A_fn = tl.construct_fn_dict(file_names_A, key_extr_pat=r'.*cnm_A_key(\d*)_(.*\.tif)\..*')


In [ ]:
to_run = sorted(dict_A_fn.keys())
print(to_run)
to_run = to_run[0:]
print(to_run)

In [ ]:
#im_sz = (512,512)
p_th = 75
down_sample = False
for k in to_run:
    print(k,dict_in_fn[k])

# Read image, iterate through slices
    im_fname = dict_in_fn[k]
    im_fname = im_fname.replace('MAX_','')
    im_fname_AT = dict_in_fn_AT[k]
    im_fname_AT = im_fname_AT.replace('MAX_','')
    print(im_fname_AT)
    im_st_fn = 'im_st_key' +  str(k) + '_' + os.path.basename(im_fname) + '.tif'
        
    print('\nReading image file from: ', im_fname)
    im = tff.imread(im_fname)
    im_shape = im.shape
    print('im', im.shape)
    im_avg = np.mean(im,axis=0)
    mask_bg = (dict_im_lab[k] < 1) & (im_avg > 0)
    im_bg = np.median(im_avg[mask_bg])
    im_bg = 0.9*im_bg
    print('im_bg estimated as ', im_bg)

    print('Loading A: ', dict_A_fn[k])
    A = np.load(os.path.join(analysis_dir, dict_A_fn[k]))
    dict_mask = {}
    
    im_sz = [im_shape[1], im_shape[2]]
    
    im_st = np.zeros((A.shape[1], im_sz[0], im_sz[1]), dtype='uint16')
    print('im_st shape', im_st.shape)
    print('Generating im_st with p_th of ', p_th)
    for i in range(A.shape[1]):
        Ai = np.copy(A[:,i])
        Ai = Ai[Ai>0]
        thr = np.percentile(Ai,p_th)
        imt = np.reshape(A[:,i],im_sz, order='F')
        im_thr = np.copy(imt)
        im_thr[im_thr<thr] = 0
        im_thr[im_thr>=thr] = i+1
        im_st[i,:,:] = im_thr
        dict_mask[i] = im_thr>0
    print('Saving im_st as ', im_st_fn)
    tff.imsave(os.path.join(analysis_dir,im_st_fn), im_st)

        
    if (down_sample):
        im = downscale_local_mean(im, (2,1,1))
        im_shape = im.shape
        print('New im', im.shape)
        
#     im = np.concatenate((im, im_AT), axis=0)

# process im
    f_dat = np.zeros((A.shape[1], im.shape[0]), dtype='float')
    print('Calculating mean fluorescence signal')
    for z in range(im.shape[0]):
        if (z%100==0):
            print('z: ', z)
#         print('im', im.shape)
        z_slice = im[z,:,:]
        for j in range(A.shape[1]):
            f_dat[j,z] = np.mean(z_slice[dict_mask[j]])
    f_dat = f_dat - im_bg
    f_dat[f_dat<0] = 0
    print('Adding a small number 2 to avoid divisions by zero in DFF calcs')
    f_dat = f_dat + 2
    print('Generating dff with perc 8 and win_sz 500')
    dff_dat = tl.convert_f_to_dff_perc(f_dat, 8, win_sz=500)
    
    post_str = str(k) + '_' + os.path.basename(im_fname) + '.npy'    
    f_fn = 'f_mean_key' +  post_str
    dff_fn = 'dff_f_mean_key' +  post_str
    print('Saving f and dff as ', f_fn, ' ', dff_fn)
    np.save(os.path.join(analysis_dir,f_fn), f_dat)
    np.save(os.path.join(analysis_dir,dff_fn), dff_dat)

# process im_AT
    print('\nReading image file from: ', im_fname_AT)
    im = tff.imread(im_fname_AT)
    im_avg = np.mean(im,axis=0)
    mask_bg = (dict_im_lab[k] < 1) & (im_avg > 0)
    im_bg = np.median(im_avg[mask_bg])
    im_bg = 0.9*im_bg
    print('im_AT_bg estimated as ', im_bg)

    if (down_sample):
        im = downscale_local_mean(im, (2,1,1))
        im_shape = im.shape
        print('New im_AT', im.shape)
    f_dat = np.zeros((A.shape[1], im.shape[0]), dtype='float')
    print('AT Calculating mean fluorescence signal')
    for z in range(im.shape[0]):
        if (z%100==0):
            print('z: ', z)
        z_slice = im[z,:,:]
        for j in range(A.shape[1]):
            f_dat[j,z] = np.mean(z_slice[dict_mask[j]])
    f_dat = f_dat - im_bg
    f_dat[f_dat<0] = 0
    print('Adding a small number 2 to avoid divisions by zero in DFF calcs')
    f_dat = f_dat + 2
    print('Generating dff with perc 8 and win_sz 500')
    dff_dat = tl.convert_f_to_dff_perc(f_dat, 8, win_sz=500)
    
#     im_st_fn = 'im_AT_st_key' +  str(k) + '_' + os.path.basename(im_fname) + '.tif'
#     print('Saving im_AT_st as ', im_st_fn)
#     tff.imsave(os.path.join(analysis_dir,im_st_fn), im_st)
#     post_str = str(k) + '_' + os.path.basename(im_fname) + '.npy'    
    f_fn = 'f_AT_mean_key' +  post_str
    dff_fn = 'dff_f_AT_mean_key' +  post_str
    print('Saving f and dff as ', f_fn, ' ', dff_fn)
    np.save(os.path.join(analysis_dir,f_fn), f_dat)
    np.save(os.path.join(analysis_dir,dff_fn), dff_dat)
